# Chapter 9. The `multiprocessing` Module [2/2]

## Queues of Work

- **프로세스간 pickle로 변환할 수 있는 오브젝트들을 보낼 수 있는 일시적인 큐를 제공해 준다.**<br>
  `multiprocessing.Queue` objects give us nonpersistent queues that can send any pickle-able objects between processes. <br>

- **object들은 반드시 pickle로 변환되고, consumer안에서 unpickle 되므로 오버헤드를 수반한다.**<br>
  They carry an overhead, as each object must be pickled to be sent and then unpickled in the consumer (along with some locking operations).<br>
  
- **저기에 드는 비용은 무시할 정도가 아님.** (not negligible)

- **하지만 워커들이 큰 작업들을 처리한다면, 커뮤니케이션 오버헤드는 감당할 만함.** (acceptable)

- **`Queue`는 native python으로 IPC를 가능하게 해 줌.** (.. ability to perfome lots of IPC ..)

- 다양한 상태(lots of state)들을 처리하는데 유용할 것임.

- 하지만 `Queue`는 지속성이 부족하기 때문에, 당신은(실패함에도 불구하고)<br>
  그들을 견고함을 요구하는 작업에 사용하는것을 원치 않을 수 있다.

In [1]:
import math
import multiprocessing
import time


FLAG_ALL_DONE = b"WORK_FINISHED"
FLAG_WORKER_FINISHED_PROCESSING = b"WORKER_FINISHED_PROCESSING"


# check prime function
def is_prime(num):

    if num % 2 == 0:
        return False

    for i in range(3, int(math.sqrt(num)) + 1, 2):
        if num % i == 0:
            return False

    return True


# define worker
def check_prime_worker(possible_primes_queue, definite_primes_queue):
    print("start worker")
    while True:

        # get recent message(or number) from queue
        n = possible_primes_queue.get()

        # if all finished stop this worker
        if n == FLAG_ALL_DONE:
            definite_primes_queue.put(FLAG_WORKER_FINISHED_PROCESSING)
            print("quit worker")
            break

        # else, find prime then put to queue.
        else:
            if is_prime(n):
                definite_primes_queue.put(n)


def main():

    primes = []

    NBR_PROCESSES = 1

    manager = multiprocessing.Manager()

    possible_primes_queue = manager.Queue()
    definite_primes_queue = manager.Queue()

    # process pool
    pool = multiprocessing.Pool(NBR_PROCESSES)
    processes = []

    for _ in range(NBR_PROCESSES):

        p = pool.Process(
            target=check_prime_worker,
            args=(possible_primes_queue, definite_primes_queue)
        )

        processes.append(p)
        p.start()

    t1 = time.time()
    number_range = range(100000000, 101000000)

    # initialize possible prime queue
    for possible_prime in number_range:
        possible_primes_queue.put(possible_prime)

    # add poison pills to stop the remote workers
    for n in range(NBR_PROCESSES):
        possible_primes_queue.put(FLAG_ALL_DONE)

    processors_indicating_they_have_finished = 0

    while True:

        # block while waiting for results
        new_result = definite_primes_queue.get()

        if new_result == FLAG_WORKER_FINISHED_PROCESSING:
            processors_indicating_they_have_finished += 1
            if processors_indicating_they_have_finished == NBR_PROCESSES:
                break
        else:
            primes.append(new_result)

    assert processors_indicating_they_have_finished == NBR_PROCESSES
    print("Took:", time.time() - t1)

    print(len(primes), primes[:10], primes[-10:])


if __name__ == '__main__':
    main()


start worker
('Took:', 58.67789006233215)
(54208, [100000007, 100000037, 100000039, 100000049, 100000073, 100000081, 100000123, 100000127, 100000193, 100000213], [100999889, 100999897, 100999901, 100999903, 100999919, 100999939, 100999949, 100999979, 100999981, 100999993])
quit worker
